In [ ]:
import numpy as np
import os
import json

directories_with_lables = [("DA Logs Benign", 0), ("DA Logs Malware", 1)]

In [ ]:
# JSON 로그를 구문 분석하는 함수
def get_API_class_method_type_from_log(log):
    # 행위 로그에서 나온 API 호출을 구문 분석한다.
    API_data_sequence = []
    with open(log) as log_file:
        json_log = json.load(log_file)
        api_calls_array = "[" + json_log["api_calls"] + "]"
        # 클래스, 메소드, API 호출 유형을 선택함
        api_calls = json.load(api_calls_array)
            for api_call in api_calls:
                data = api_call['class'] + ":" + api_call["method"] + ":" + api_call["type"]
                API_data_sequence.append(data)
        return API_data_sequence

In [ ]:
# 로그를 Corpus로 읽고 레이블을 수집함
data_corpus = []
labels = []
for directory, label in directories_with_lables:
    logs = os.listdir(directory)
    for log_path in logs:
        file_path = directory + "/" + log_path
        try:
            data_corpus.append(get_API_class_method_type_from_log(file_path))
            labels.append(label)
        except:
            pass

# 말뭉치 안에 있는 데이터 확인하기
print(data_corpus[0])

In [ ]:
# 훈련 데이터와 테스트 데이터를 분할한다.
from sklearn.model_selection import train_test_split

corpus_train, corpus_test, y_train, y_test = train_test_split(
    data_corpus, labels, test_size=0.2, random_state=11
)

In [ ]:
# 현재 데이터 형식을 수정하는, N-Gram 추출 함수를 불러옴
import collections
from nltk import ngrams
import numpy as np

def read_file(file_path):
    # 이진 파일의 문자열을 읽는다.
    with open(file_path, "rb") as binary_file:
        data = binary_file.read()
    return data

def text_to_Ngrams(text, n):
    # 텍스트에서 N-그램 리스트를 만든다.
    Ngrams = ngrams(text, n)
    return list(Ngrams)

def get_Ngram_counts(text, N):
    # 텍스트에서 N-그램의 빈도수 갯수를 얻는다.
    Ngrams = text_to_Ngrams(text, N)
    return collections.Counter(Ngrams)

In [ ]:
# N-그램을 수집한다.
N = 4
total_Ngram_count = collections.Counter([])
for file in corpus_train:
    total_Ngram_count += get_Ngram_counts(file, N)
    
# 가장 빈도수가 높은 N-그램(K1)을 3000개로 좁힌다.
K1 = 3000
K1_most_frequent_Ngrams = total_Ngram_count.most_common(K1)
K1_most_frequent_Ngrams_list = [x[0] for x in K1_most_frequent_Ngrams]
K1_most_frequent_Ngrams_list[:7]

In [ ]:
# 샘플들을 N-그램 카운트 벡터로 특성화하는 함수를 만든다.
def featurize_sample(file, Ngram_list):
    # 샘플의 특성 벡터를 만든다. 특성은 우리가 선택한 N-그램 K1개의 갯수다.
    K1 = len(Ngram_list)
    feature_vector = K1 * [0]
    fileNgrams = get_Ngram_counts(file, N)
    for i in range(K1):
        feature_vector[i] = fileNgrams[Ngram_list[i]]
    return feature_vector

# 위 함수를 훈련 샘플과 테스트 샘플을 특성화하는데 적용한다.
X_train = []
for sample in corpus_train:
    X_train.append(featurize_sample(sample, K1_most_frequent_Ngrams_list))
X_train = np.asarray(X_train)

X_test = []
for sample in corpus_train:
    X_test.append(featurize_sample(sample, K1_most_frequent_Ngrams_list))
X_test = np.asarray(X_test)

In [ ]:
# 상호정보를 사용해 가장 빈도수가 높은 N-그램(K1=3000개)을 가장 많은 정보를 가진 N-그램(K2=500개)로 좁힌다. 그런 다음 파이프라인을 설정해 XGBoost 분류기를 실행한다.
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier

K2 = 500
mi_pipeline = Pipeline(
    [
        ("mutual_information", SelectKBest(mutual_info_classif, k=K2)),
        ("xgb", XGBClassifier())
    ]
)

# 파이프라인을 훈련하고, 정확도를 평가함.
mi_pipeline.fit(X_train, y_train)
print("훈련 정확도: ",mi_pipeline.score(X_train,y_train))
print("테스트 정확도: ",mi_pipeline.score(X_test,y_test))